<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2017</h1>
<h1 style="font-size: 250%;">Problem Set #2 </h1>
<h2>Version 2</h2>
<h1>Issued Monday, 2/20/2017; Due Monday, 3/6/2017</h1>
</div>

**Homework Information:** Some of the problems are probably too long to complete the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Zhilan"
    last = "Zhou"
    onyen = "zzl"
    pid = "720432337"

---
**Problem #1:** Consider the table of integer molecular weights for amino acid residues that we have used in class and is repeated below.

        Daltons = { 
            'A':  71, 'C': 103, 'D': 115, 'E': 129,
            'F': 147, 'G':  57, 'H': 137, 'I': 113,
            'K': 128, 'L': 113, 'M': 131, 'N': 114,
            'P':  97, 'Q': 128, 'R': 156, 'S':  87,
            'T': 101, 'V':  99, 'W': 186, 'Y': 163 
        }

Which peptide *combinations* have a total molecular weight of 555 daltons? A *combination* means that the order of the peptides is unimportant (i.e. "AKS" is the same combination as any of "ASK", "KAS", "KSA", "SAK", and "SKA"). When reporting your answer you should lexigraphically sort the resides within each combination ("AKS" is the sorted version of the previous example). Furthermore, the list of combinations should also be lexigraphically sorted (The python sorted() method will do this for you).

---
**Problem #2:** The protein for initiating chromosome replication, *DnaA*, which we discussed in class, appears in the genome of [*Brevibacillus brevis*](http://csbio.unc.edu/mcmillan/Comp555S17/data/BacillusBrevis.fa). Its peptide sequence is: 

         DnaA = "MDAAISELWRKVLAKIEKSLSKPSFDTWLKATKATTLEEDALIVVAPNDFARDWLETRYA" +\
                "QLITDTLYEVTGINMKVKFVALQNPDAAFADEQPVPRVKMSEPPTVADDQPPSILNPKYT" +\
                "FDTFVIGSGNRFAHAASLAVAEAPAKAYNPLFIYGGVGLGKTHLMHAIGHYVIQHNPSAK" +\
                "VVYLSSEKFTNEFINSIRDNKAVEFRNKYRSVDVLLIDDIQFLAGKESTQEEFFHTFNAL" +\
                "HEESKQIIISSDRPPKEIPTLEDRLRSRFEWGLITDIQPPDLETRIAILRKKAKAENLDI" +\
                "PNEVMAYIANQIDSNIRELEGALIRVVAYSSLINRDIDTQLAAEALKDIIPSSRPRVITI" +\
                "IDIQRTVGEAFSLKLEDFKAKKRTKTVAFPRQIAMYLSRELTDASLPKIGDEFGGRDHTT" +\
                "VIHAHEKISRALANDPHMQTTIQSLIEKLKANH"
       
How many possible DNA-sequence encodings are possible for this protein? (**Note:** you should consider all possible STOP codons in your calculations)

---
**Problem #3:** Find the coding sequence for the *DnaA* gene in [*Brevibacillus brevis*](http://csbio.unc.edu/mcmillan/Comp555S17/data/BacillusBrevis.fa). Your answer should give the sequence's slice interval in the genome (i.e. seq[start:end], where end=start+3&times;454 <strike>28</strike>). Note the the gene sequence might appear either on the primary sequence or its reverse complement. ***Hint:*** Many procaryotic proteins make use of alternate start codons instead of the usual "AUG" (Methionine) sequence. These alternate start codes are still translated to Methionine, but they do not appear as the expected codon in the DNA sequence. *DnaA* is such a protein, thus you will not find it if you expect it to start with "ATG".

**Problem #4:** How many *distinct* subpeptides can the *DnaA* protein be broken into? How many *distinct* molecular weights appear in its Theoretical Spectrum?

**Problem #5: Programming Problem**

Given a collection of strings, a larger string containing every one of the smaller strings as a substring is called a superstring. Assuming parsimony, the shortest possible superstring over a collection of reads serves as a candidate chromosome.

Write a program that inputs at most 50 DNA strings <strike>whose lengths do not exceed 100 bp</strike> in FASTA format (which represent reads deriving from sequencing) and outputs a shortest superstring containing all the given strings. The dataset is guaranteed to satisfy the following condition: there exists a unique way to reconstruct the entire chromosome from these reads by gluing together pairs of reads that overlap by at least 9 basepairs <strike>half their length</strike>. You can assume that all strings are properly *stranded*, meaning that there is no need to consider reverse complements when assembling your path.

Sample Dataset:

        >Rosalind_56
        ATTAGACCTG
        >Rosalind_57
        CCTGCCGGAA
        >Rosalind_58
        AGACCTGCCG
        >Rosalind_59
        GCCGGAATAC
       
       
Sample Output (first line only, the following four lines are for illustration purposes):

      
                     ATTAGACCTGCCGGAATAC
        Rosalind_56  ----------
        Rosalind_57        ----------
        Rosalind_58     ----------
        Rosalind_59           ----------

Another sample dataset can be downloaded [*here*](http://csbio.unc.edu/mcmillan/Comp555S17/data/PS2contigs.fa).

In [2]:
import itertools

def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'rb')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+'+''.join(lines))
    return (headers, sequences)

hdr, seq = loadFasta("PS2contigs.fa")

class BasicGraph:
    def __init__(self, vlist=[]):
        """ Initialize a Graph with an optional vertex list """
        self.index = {v:i for i,v in enumerate(vlist)}    # looks up index given name
        self.vertex = {i:v for i,v in enumerate(vlist)}   # looks up name given index
        self.edge = []
        self.edgelabel = []
    def addVertex(self, label):
        """ Add a labeled vertex to the graph """
        index = len(self.index)
        self.index[label] = index
        self.vertex[index] = label
    def addEdge(self, vsrc, vdst, label='', repeats=True):
        """ Add a directed edge to the graph, with an optional label. 
        Repeated edges are distinct, unless repeats is set to False. """
        e = (self.index[vsrc], self.index[vdst])
        if (repeats) or (e not in self.edge):
            self.edge.append(e)
            self.edgelabel.append(label)
            
class ImprovedGraph(BasicGraph):
    def SearchTree(self, path, verticesLeft):
        """ A recursive Branch-and-Bound Hamiltonian Path search. 
        Paths are extended one node at a time using only available
        edges from the graph. """
        if (len(verticesLeft) == 0):
            self.PathV2result = [self.vertex[i] for i in path]
            return True
        for v in verticesLeft:
            if (len(path) == 0) or ((path[-1],v) in self.edge):
                if self.SearchTree(path+[v], [r for r in verticesLeft if r != v]):
                    return True
        return False
    def hamiltonianPath(self):
        """ A wrapper function for invoking the Branch-and-Bound 
        Hamiltonian Path search. """
        self.PathV2result = []
        self.SearchTree([],sorted(self.index.values()))                
        return self.PathV2result

def overlap(suffix, prefix):
    N = 0
    for i in xrange(min(len(suffix)-1, len(prefix)-1),8,-1):
        if suffix[-i:] == prefix[1:i+1]:
            N = i
            break
    return N

binary = [base for base in xrange(len(hdr))]

G1 = ImprovedGraph(binary)
for i in xrange(len(hdr)):
    for j in xrange(len(hdr)):
        if (i == j):
            continue
        edge = overlap(seq[i], seq[j])
        if (edge > 0):
            G1.addEdge(i,j)
            
path = G1.hamiltonianPath()

superstring = seq[path[0]][1:]

for i in xrange(1, len(hdr)):
    superstring += seq[path[i]][1+(overlap(seq[path[i-1]], seq[path[i]])):]

print superstring

CATGATCATCATCATCATCATCATCATCAGCATCGTCATCATCATCATCATCATCATGATCATCATCATCAGCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCGTCATCAGCATCATCATCATCGTCGGCATGAGCATCATCATCGTCATGATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCGTCATCGTCATGGTCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCGTCATCGGCATCATCATCATCATCATCATCATCGTCATCATCATCATCATCATCATCATCATCATGATCATCATCATGATCAGCGTCATCATCATCATCATCATCATCAGCATCATCATCATCGTCATCATGATCATCATCATCATCATCATGAGCATCATCATCAGCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATCATGATCATCATCATCATCATCATGATCATCATCATCGTCAGCATCATCATCATCATCATCATCATCATCATGATCATCGTCATCATCATCATCATCATCATCATCATCATCATCATGATCATGATCATGGTCATCATGATCATCATCATCATCATCATCATCATCATCATCATGATCATCATCATCATCATCATCAGCATCGGCATCATCATCATCATCATGGTCATCATCATCATCATCGTCATGATCATCATCATCATCATCATCAGCATCATCATCATCAGCAGCATCATCATCATCGTGATCATCATCGTCATCATCATCATCATCATCATCATCATCATCAGGATCATCATCATCATCATCATCGTCATGATCATCATCATCATCATCGTCATCATCATCATCAGCATCATCATCAGCATCATCATCATCATCATCAGCATCATCATCATCAGG

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set. **Make sure that you do not overwrite a previous problem set!**